In [1]:
import os
os.environ["OMP_NUM_THREADS"]='1'
import osmnx as ox
import networkx as nx
from datetime import datetime

import pandas as pd
import geopandas as gpd

from pyproj import Transformer 
import numpy as np
import scipy
from descartes import PolygonPatch

import matplotlib.pyplot as plt
from matplotlib import *
from matplotlib.patches import *
from matplotlib.lines import Line2D
import matplotlib.colors as mcolors

import contextily as cx #Copyright (c) 2016, Dani Arribas-Bel. All rights reserved.

import math
import itertools
from shapely.geometry import MultiPolygon, Polygon, Point, LineString
from shapely.ops import nearest_points
from IPython.display import Image, IFrame

from sklearn import cluster
from sklearn.cluster import KMeans


import pandapower as pp
import pandapower.timeseries as ts
import pandapower.control as control
from pandapower.plotting import simple_plot, simple_plotly, pf_res_plotly 
import pandapower.plotting.plotly as plotly


In [2]:
def delete_all_components(net):
    # Alle Busse löschen
    net.bus.drop(net.bus.index, inplace=True)
    
    # Alle Erzeugungen löschen
    net.gen.drop(net.gen.index, inplace=True)
    net.sgen.drop(net.sgen.index, inplace=True)
    net.storage.drop(net.storage.index, inplace=True)

    # Alle Lasten löschen
    net.load.drop(net.load.index, inplace=True)
    
    # Alle Leitungen löschen
    net.line.drop(net.line.index, inplace=True)
    
    # Alle externen Netze löschen
    net.ext_grid.drop(net.ext_grid.index, inplace=True)

In [3]:
def initialize_network():
    # Erstellen eines neuen leeren Netzwerks
    return pp.from_json('Daten/Netzmodell_PandaPower_Weilimdorf_V2.json')

In [4]:
def extract_results(net):
    # Maximale Belastung der Kabel (in %)
    max_line_loading = net.res_line.loading_percent.max()
    
    # Maximale Belastung der Trafos (in %)
    if len(net.trafo) > 0:
        max_trafo_loading = net.res_trafo.loading_percent.max()
    else:
        max_trafo_loading = 0  # Falls keine Trafos im Netz sind

    # Gesamte Erzeugungsleistung (in MW)
    total_generation = net.res_gen.p_mw.sum() + net.res_sgen.p_mw.sum() + net.res_storage.p_mw.sum()
    
    # Gesamte Last (in MW)
    total_load = net.res_load.p_mw.sum()
    
    # Weitere interessante Parameter
    max_bus_voltage = net.res_bus.vm_pu.max()
    min_bus_voltage = net.res_bus.vm_pu.min()
    total_losses = net.res_line.pl_mw.sum() + net.res_trafo.pl_mw.sum()

    return {
        "max_line_loading": max_line_loading,
        "max_trafo_loading": max_trafo_loading,
        "total_generation": total_generation,
        "total_load": total_load,
        "max_bus_voltage": max_bus_voltage,
        "min_bus_voltage": min_bus_voltage,
        "total_losses": total_losses
    }

In [5]:
def find_rings(net):
    visited_lines = set()
    rings = []

    for line_idx in net.line.index:
        if line_idx in visited_lines:
            continue
        
        # Fange einen neuen Ring an
        stack = [line_idx]
        ring_lines = set()
        ring_buses = set()

        while stack:
            current_line = stack.pop()
            
            if current_line in visited_lines:
                continue
            
            visited_lines.add(current_line)
            ring_lines.add(current_line)
            
            # Füge die Busse der aktuellen Leitung hinzu
            from_bus = net.line.at[current_line, 'from_bus']
            to_bus = net.line.at[current_line, 'to_bus']
            ring_buses.update([from_bus, to_bus])

            # Finde benachbarte Leitungen (die zu den aktuellen Bussen führen)
            connected_lines = net.line[((net.line['from_bus'] == from_bus) | 
                                        (net.line['to_bus'] == from_bus) |
                                        (net.line['from_bus'] == to_bus) | 
                                        (net.line['to_bus'] == to_bus))].index
            
            # Füge alle benachbarten Leitungen, die noch nicht besucht wurden, zum Stack hinzu
            for next_line in connected_lines:
                if next_line not in visited_lines:
                    stack.append(next_line)
        
        if ring_lines:
            rings.append((ring_buses, ring_lines))

    return rings

In [6]:
def export_network_parameters(net):
    # Erstelle eine Liste, um die Parameter zu speichern
    data = []

    # Gesamtanzahl der Busse
    total_buses = len(net.bus)
    data.append(['total_buses', total_buses])
    
    # Anzahl der Verbraucher
    total_loads = len(net.load)
    data.append(['total_loads', total_loads])

    # Anzahl der Transformatoren (Ortsnetzstationen)
    total_transformers = len(net.trafo)
    data.append(['total_transformers', total_transformers])
    
    # Gesamtanzahl der Leitungen
    total_lines = len(net.line)
    data.append(['Anzahl Leitungen', total_lines])

    # Gesamtlänge aller Leitungen
    total_line_length_km = net.line['length_km'].sum()
    data.append(['Gesamtlänge Leitungen (km)', total_line_length_km])
    
    # Finde alle Ringe im Netz
    rings = find_rings(net)

    # Anzahl der erkannten Ringe
    total_rings = len(rings)
    data.append(['Anzahl Ringe', total_rings])
    data.append(['Durchschnittliche Leitungslänge pro Ring', total_line_length_km/total_rings])

    # Optional: Details über jeden Ring
    for i, (buses, lines) in enumerate(rings):
        data.append([f'Leitungslänge Ring {i} (km)', net.line.loc[list(lines), 'length_km'].sum()])
    
    # Konvertiere die Liste in ein DataFrame
    df_parameters = pd.DataFrame(data, columns=['Parameter', 'Value'])
    
    return df_parameters

In [7]:
net = initialize_network()
param = export_network_parameters(net)
delete_all_components(net)
param

,Parameter,Value
0,total_buses,380.000000
1,total_loads,75.000000
2,total_transformers,2.000000
3,Anzahl Leitungen,294.000000
4,Gesamtlänge Leitungen (km),21.986507
5,Anzahl Ringe,6.000000
6,Durchschnittliche Leitungslänge pro Ring,3.664418
7,Leitungslänge Ring 0 (km),2.869000
8,Leitungslänge Ring 1 (km),2.060699
9,Leitungslänge Ring 2 (km),6.505334


In [8]:

def lade_messdaten(net, P):
    #Wird nur einmal benötigt

    # Erstelle einen leeren DataFrame, der später gefüllt wird
    df = pd.DataFrame()
    
    # Durchlaufe alle Busse im Netzwerk
    for bus in net.bus.name:
        # Konvertiere den Bus-Index in einen String, falls es sich um eine Zahl handelt
        bus_str = str(bus)

        # Extrahiere die Nummer des Busses aus dem Namen (z.B. "AbA 3300" -> "3300")
        bus_nummer = ''.join(filter(str.isdigit, bus_str))
        if len(bus_nummer) != 4:
            continue
        # Suche nach dem Ordner, der zu dieser Bus-Nummer gehört
        messordner = None
        for ordner in os.listdir('./Messungen'):
            ordnerpfad = os.path.join('./Messungen', ordner)

            if os.path.isdir(ordnerpfad) and bus_nummer in ordner:  # Überprüfen, ob es sich um einen Ordner handelt
                messordner = ordnerpfad
                break

        # Falls kein passender Ordner gefunden wurde, überspringe diesen Bus
        if not messordner:
            continue
        
        # Durchsuche den gefundenen Ordner nach der Datei, die mit der Busnummer beginnt
        for datei in os.listdir(messordner):
            if datei.startswith(bus_nummer) and datei.endswith(P + '.xlsx'):
                # Lade die Excel-Datei
                dateipfad = os.path.join(messordner, datei)
                df_temp = pd.read_excel(dateipfad, usecols=[2], header=0, names=[bus])
                
                # Füge die Daten dem DataFrame hinzu
                if df.empty:
                    df = df_temp
                else:
                    df = pd.concat([df, df_temp], axis=1)
                break
    
    return df

In [9]:
def lade_messdaten_normal(net,type):
    if type == "P+":
        return pd.read_excel('./Messungen/p_messungen_1.xlsx')
    elif type == "Q1":
        return pd.read_excel('./Messungen/q_messungen_1.xlsx')
    elif type == "P-":
        return pd.read_excel('./Messungen/p_gen_messungen_1.xlsx')
    else:
        print("Fehler")

In [10]:
net = initialize_network()

result_df_p = lade_messdaten(net,"P+")
result_df_q = lade_messdaten(net,"Q1")
result_df_p_gen = lade_messdaten(net,"P-")
result_df_p.to_excel('./Messungen/p_messungen_1.xlsx')
result_df_q.to_excel('./Messungen/q_messungen_1.xlsx')
result_df_p_gen.to_excel('./Messungen/p_gen_messungen_1.xlsx')

delete_all_components(net)

Last_Sammelschiene = pd.read_csv("./Messungen/WIDRF_2022_P.csv", sep=';',decimal=",", encoding='utf-8')

Spalten = ["K17","K27","K09","K29","K18","K26"]

Last_Sammelschiene_Winter = Last_Sammelschiene.iloc[1537:2209][Spalten]
Last_Sammelschiene_Fruehling = Last_Sammelschiene.iloc[10177:10849][Spalten]
Last_Sammelschiene_Sommer = Last_Sammelschiene.iloc[22177:22849][Spalten]

Last_Ring_1 = -Last_Sammelschiene_Winter["K17"] - Last_Sammelschiene_Winter["K27"]
Last_Ring_2 = -Last_Sammelschiene_Winter["K09"] - Last_Sammelschiene_Winter["K29"]
Last_Ring_3 = -Last_Sammelschiene_Winter["K18"] - Last_Sammelschiene_Winter["K26"]

In [11]:
messdaten_p_Winter = result_df_p[1537:2209]
messdaten_p_gen_Winter = result_df_p_gen[1537:2209]
messdaten_q_Winter = result_df_q[1537:2209]

messdaten_p_Sommer = result_df_p[22177:22849]
messdaten_p_gen_Sommer = result_df_p_gen[22177:22849]
messdaten_q_Sommer = result_df_q[22177:22849]

messdaten_p_Fruehling = result_df_p[10177:10849]
messdaten_p_gen_Fruehling = result_df_p_gen[10177:10849]
messdaten_q_Fruehling = result_df_q[10177:10849]


In [12]:
print(messdaten_p_Winter.iloc[671])
print(Last_Sammelschiene_Winter.iloc[671])

AbA 3300     10.320
AbA 3580     30.437
AbA 3491     12.400
AbA 2818      8.500
AbA 3953      5.520
AbA 3429      9.080
AbA 3209      3.940
AbA 2431     30.280
AbA 2559      2.000
AbA 3735     27.000
AbA 3830     56.120
AbA 3558     63.580
AbA 3689    165.008
AbA 2053     50.520
AbA 3295      2.952
AbA 2873      4.520
AbA 3552     15.240
AbA 2259     20.980
AbA 3007     18.288
AbA 2418      3.968
AbA 4156      8.560
AbA 3867      7.560
AbA 4177      1.440
AbA 4187     23.520
AbA 3647     67.720
AbA 3368     10.380
AbA 2751     24.235
AbA 2931     10.560
AbA 2675     27.312
AbA 2441     26.420
AbA 3432     44.280
AbA 3484     23.720
AbA 2278     49.360
AbA 3584     12.000
AbA 3470      9.792
AbA 3576     38.040
AbA 3147     64.000
AbA 2981     28.840
AbA 3352     27.000
AbA 3114     87.200
AbA 3247    158.960
AbA 3972    270.000
AbA 2382     28.400
AbA 2913     27.600
AbA 3226     47.960
AbA 3118     18.800
Name: 2208, dtype: float64
K17   -0.607
K27    0.000
K09   -0.514
K29   -0.365
K

In [13]:
net2 = initialize_network()
def find_and_print_connected_buses_from_first_line(net):
    # 1. Löschen von Leitungen mit in_service = False
    net.line.drop(net.line[net.line.in_service == False].index, inplace=True)

    # Set zum Speichern der besuchten Busse
    visited_buses = set()
    # Liste zum Speichern der Ringe
    rings = []

    # 2. Tiefensuche (DFS) ab einem Bus, um alle verbundenen Busse zu finden
    def dfs(bus, current_ring):
        if bus in visited_buses:
            return
        visited_buses.add(bus)
        current_ring.append(bus)  # Füge den aktuellen Bus zum aktuellen Ring hinzu
        #print(f"Besuchter Bus: {bus}")

        # Finde alle verbundenen Busse über Leitungen
        for _, line in net.line.iterrows():
            if line.in_service:
                if line.from_bus == bus and line.to_bus not in visited_buses:
                    dfs(line.to_bus, current_ring)
                elif line.to_bus == bus and line.from_bus not in visited_buses:
                    dfs(line.from_bus, current_ring)

    # 3. Finde die erste in Betrieb befindliche Leitung und starte die Suche
    for _, line in net.line.iterrows():
        if line.in_service:
            start_bus = line.from_bus
            #print(f"Starte Suche ab erstem Bus der ersten Leitung: {start_bus}")
            current_ring = []  # Erstelle einen neuen Ring für die aktuelle Suche
            dfs(start_bus, current_ring)
            rings.append(current_ring)  # Speichere den aktuellen Ring
            break  # Nach der ersten Leitung aufhören und mit den restlichen Bussen fortfahren

    # 4. Starte neue Suchen für jeden nicht besuchten Bus
    for bus in net.bus.index:
        if bus not in visited_buses:
            #print(f"\nStarte neue Suche ab Bus: {bus}")
            current_ring = []  # Erstelle einen neuen Ring für die aktuelle Suche
            dfs(bus, current_ring)
            rings.append(current_ring)  # Speichere den aktuellen Ring
    # 5. Entferne Ringe mit nur einem Element
    rings = [ring for ring in rings if len(ring) > 1]

    # Ausgabe der Ringe
    #print("\nGesammelte Ringe:")
    for i, ring in enumerate(rings, 1):
        print(f"Ring {i}: {ring}")

    return rings

rings = find_and_print_connected_buses_from_first_line(net2)

lasten_summe = []
#lasten_summe1 = []
for ring in rings:
    ring_last_sum = 0
    #ring_last_sum1 = 0  # Initialisiere die Summe für den aktuellen Ring
    for bus in ring:
        # Summiere die Lasten für jeden Bus im Ring
        ring_last_sum += net2.load.loc[net2.load.bus == bus, 'p_mw'].sum()  # Leistung in MW
        #ring_last_sum1 += net1.load.loc[net1.load.bus == bus, 'p_mw'].sum()  # Leistung in MW
    lasten_summe.append(ring_last_sum)
    #lasten_summe1.append(ring_last_sum1)

print(lasten_summe)
#print(lasten_summe1)

Ring 1: [34, 1, 223, 222, 221, 220, 39, 261, 260, 259, 258, 257, 40, 237, 236, 235, 234, 233, 43, 239, 240, 241, 242, 243, 244, 245, 246, 247, 33, 272, 32, 172, 82]
Ring 2: [4, 171, 170, 169, 168, 167, 166, 165, 164, 163, 162, 161, 160, 159, 158, 157, 156, 155, 154, 153, 118]
Ring 3: [9, 194, 193, 192, 191, 190, 189, 188, 187, 186, 185, 184, 183, 119]
Ring 4: [26, 212, 211, 210, 38, 249, 248, 28, 219, 54, 216, 56, 203, 202, 201, 200, 199, 198, 197, 196, 195, 37, 250, 251, 252, 27, 230, 229, 228, 227, 226, 225, 224, 41, 231, 232, 42, 180, 179, 178, 177, 176, 175, 174, 173, 88, 238, 36, 204, 205, 206, 207, 208, 209, 35, 253, 254, 255, 256, 29, 279, 278, 277, 276, 275, 55, 274, 273, 44, 218, 217, 51, 281, 45, 50, 280, 122, 288, 287, 47, 286, 285, 46, 282, 283, 52, 284, 53, 215, 214, 213, 48, 268, 269, 270, 271, 49, 264, 263, 262, 31, 265, 266, 267, 30, 182, 181, 68]
Ring 5: [72, 308, 309, 310, 311, 312, 313, 131, 314, 315, 316, 317, 318, 319, 320, 321, 322, 132, 323, 324, 133, 325, 326, 3

In [14]:
def update_loads(net, load_p, load_q, generation,Last_Ring_1,Last_Ring_2,Last_Ring_3):
    # Alle Lasten löschen
    net.load.drop(net.load.index, inplace=True)
    net.sgen.drop(net.sgen.index, inplace=True)

    if isinstance(load_p, pd.Series):
        load_df_p = load_p.to_frame()
    else:
        load_df_p = load_p

    if isinstance(load_q, pd.Series):
        load_df_q = load_q.to_frame()
    else:
        load_df_q = load_q

    if isinstance(generation, pd.Series):
        gen_df = generation.to_frame()
    else:
        gen_df = generation


    for bus in net.bus.name:
        bus_name = str(bus)
        Flag = 0
        if bus_name in load_df_p.index:
            try:
                # Werte für P und Q holen
                P = load_df_p.loc[bus_name].iloc[0]
                if bus_name in load_df_q.index:
                    Q = load_df_q.loc[bus_name].iloc[0] 
                else:
                    Q=0

                if bus_name in gen_df.index:
                    gen = gen_df.loc[bus_name].iloc[0]
                    pp.create_sgen(net,bus_idx,p_mw = gen, name = bus_name)


                # Den Bus-Index im Pandapower-Netzwerk finden
                bus_idx = net.bus.index[net.bus['name'] == bus_name].tolist()
                if bus_idx:
                    bus_idx = bus_idx[0]  # Den ersten (und wahrscheinlich einzigen) Bus-Index verwenden
                    pp.create_load(net, bus_idx, p_mw=P, q_mvar=Q, name = bus_name)
              
            except KeyError:
                print(f"Spalte für '{bus_name}' nicht gefunden.")

    

In [15]:
results = []
plot = False
P_Zeitreihe = messdaten_p_Winter
Q_Zeitreihe = messdaten_q_Winter
Gen_Zeitreihe = messdaten_p_gen_Winter


for j in range(len(P_Zeitreihe)):
    p = P_Zeitreihe.iloc[j]/1000
    q = Q_Zeitreihe.iloc[j]/1000
    gen = Gen_Zeitreihe.iloc[j]/1000

    if j%10 == 0:
        print(f"Iteration {j}/{len(P_Zeitreihe)}")
    
    # Netzwerk initialisieren
    if j == 0:
        net1 = initialize_network()
    
    update_loads(net1,p,q,gen,Last_Ring_1,Last_Ring_2,Last_Ring_3)
    
    # Power Flow Berechnung durchführen
    pp.runpp(net1, algorithm='nr', error_tolerance_vm_pu=1e-08, symmetric=True, validate_input=False)

    # Ergebnisse speichern 
    result_data = extract_results(net1)
    result_data.update({
        "Zeitschritt": j,
    })
    results.append(result_data)

    if j%48 == 0 and plot:
        pp.plotting.pf_res_plotly(net1)    
    
    #Ergebnisse speichern
    if j == len(P_Zeitreihe) - 1:
        current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        file_name_net = f"./results/Weilimdorf_{current_time}_pp_net.xlsx"
        file_name_result = f"./results/Weilimdorf_{current_time}_pp_net_result.xlsx"
        df = pd.DataFrame(results)
        pp.to_excel(net1, file_name_net)
        df.to_excel(file_name_result)
        break  # Abbrechen der Schleife vor der letzten Iteration
    

Iteration 0/672
Iteration 10/672
Iteration 20/672
Iteration 30/672
Iteration 40/672
Iteration 50/672
Iteration 60/672
Iteration 70/672
Iteration 80/672
Iteration 90/672
Iteration 100/672
Iteration 110/672
Iteration 120/672
Iteration 130/672
Iteration 140/672
Iteration 150/672
Iteration 160/672
Iteration 170/672
Iteration 180/672
Iteration 190/672
Iteration 200/672
Iteration 210/672
Iteration 220/672
Iteration 230/672
Iteration 240/672
Iteration 250/672
Iteration 260/672
Iteration 270/672
Iteration 280/672
Iteration 290/672
Iteration 300/672
Iteration 310/672
Iteration 320/672
Iteration 330/672
Iteration 340/672
Iteration 350/672
Iteration 360/672
Iteration 370/672
Iteration 380/672
Iteration 390/672
Iteration 400/672
Iteration 410/672
Iteration 420/672
Iteration 430/672
Iteration 440/672
Iteration 450/672
Iteration 460/672
Iteration 470/672
Iteration 480/672
Iteration 490/672
Iteration 500/672
Iteration 510/672
Iteration 520/672
Iteration 530/672
Iteration 540/672
Iteration 550/672
Ite